## **Using Retrieval Chain**

In [26]:
from langchain_mistralai import ChatMistralAI
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import WebBaseLoader
from langchain_mistralai import MistralAIEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain_community.document_loaders import WebBaseLoader 
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS

import getpass
import os
from dotenv import load_dotenv

load_dotenv()

True

In [27]:
# Load the API key from the environment
def get_mistral_api_key():
    key = os.getenv("MISTRAL_API_KEY")
    if not key:
        key = getpass.getpass("Enter your Mistral API key: ")
        os.environ["MISTRAL_API_KEY"] = key
    return key

# Get the Mistral API key
mistral_api_key = get_mistral_api_key()

In [28]:
URL1="https://techcrunch.com/2024/03/04/anthropic-claims-its-new-models-beat-gpt-4/"
URL2="https://techcrunch.com/2024/03/28/ai21-labs-new-text-generating-ai-model-is-more-efficient-than-most/"

In [29]:
loader = WebBaseLoader([URL1,URL2])

In [30]:
data = loader.load()

In [31]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
chunks = text_splitter.split_documents(data)

In [32]:
# embeddings = MistralAIEmbeddings(
#     model="mistral-embed",
# )
embeddings = OllamaEmbeddings(model='nomic-embed-text:latest')

### Using in-memory vector db FAISS

In [33]:
vector = FAISS.from_documents(chunks, embeddings)
retriever = vector.as_retriever()

In [34]:
prompt_template = """
Answer the question {input} based solely on the context below:
\n\n'<context>\n{context}\n</context>'
If you can't find an answer, say I don't know.
"""

In [35]:
prompt = PromptTemplate.from_template(prompt_template)

In [36]:
llm = ChatMistralAI(model="mistral-small-2501", temperature=0, max_retries=0)

In [37]:
combine_docs_chain = create_stuff_documents_chain(llm, prompt)

In [38]:
chain = create_retrieval_chain(retriever, combine_docs_chain)

In [41]:
result = chain.invoke({"input": "List the models and their token size"})

In [42]:
print(result['answer'])

Based on the context provided, the models and their token sizes are:

1. **GPT-4** - Token size not specified.
2. **Gemini 1.5 Pro** - 1 million tokens.
